# Comparação das Casas

Aqui vamos utilizar a estrategia mostrada no artigo:
https://medium.com/swlh/how-to-make-money-from-web-scraping-without-selling-data-92c1f961b25

Regra principal:
 - (1/odds1) + (1/odds2) < 1.0

In [52]:
import pandas as pd
from fuzzywuzzy import process

In [39]:
# Carregando os datasets
betano = pd.read_csv('dados/dados tratados/jogos_betano_tratado.csv')
betfair = pd.read_csv('dados/dados tratados/jogos_betfair_tratado.csv')
sporting = pd.read_csv('dados/dados tratados/jogos_sportingbet_tratado.csv')

In [19]:
betano.head()

,Unnamed: 0,data,hora,time_casa,time_visitante,casa_ganha,empate,visitante_ganha,maisq,menosq,simodd,nãoodd
0,0,1900-01-30,22:00,Club Atletico Tucuman II,Boca Juniors II,3.90,3.50,1.82,1.98,1.65,1.88,1.72
1,1,1900-01-30,22:00,CF Tigres UANL (F),Deportivo Toluca FC (F),1.07,10.00,14.50,NaN,NaN,NaN,NaN
2,2,1900-01-30,22:10,Once Caldas,Deportes Tolima,2.32,2.92,3.60,2.70,1.47,2.15,1.65
3,3,1900-01-31,00:00,Mazatlán FC (F),Cruz Azul (F),6.50,4.75,1.36,1.52,2.20,NaN,NaN
4,4,1900-01-31,00:05,Guadalajara Chivas (F),Juarez (F),1.42,4.70,5.50,1.60,2.05,NaN,NaN


In [20]:
betfair.head()

,Unnamed: 0,data/hora,maisq25,menosq25,casa_ganha,empate,visitante_ganha,time_casa,time_visitante
0,0,72′,0,2.0,1.68,2.1,25.0,1.1,Roma
1,1,27′,3,0.0,100.00,250.0,Sport Recife,NaN,NaN
2,2,16′,1,0.0,1.52,2.4,1.28,10.0,Sporting CP
3,3,74′,0,2.0,1.90,1.82,250.0,1.001,Montpellier
4,4,75′,2,1.0,1.10,8.0,50.0,Monaco,Auxerre


In [21]:
sporting.head()

,Unnamed: 0,time_casa,time_visitante,aovivo,casa_ganha,empate,visitante_ganha,mais/menosQ,maisQ,menos_que
0,0,Manchester United,Nottingham Forest,AO VIVO,2º T 75:37,2.0,0.00,Aposte agora,NaN,NaN
1,1,Lens,Nice,AO VIVO,2º T 75:47,0.0,1.00,11.50,3.50,1.40
2,2,Roma,Cremonese,AO VIVO,2º T 72:37,0.0,2.00,18.50,7.25,1.12
3,3,Sporting,Braga,AO VIVO,1º T 17:27,1.0,0.00,1.29,4.60,9.00
4,4,Botafogo,Nova Iguaçu,Começando em 25 min,1.55,3.8,6.25,"2,5",2.05,1.68


### Tirando as colunas que não serão usadas

In [40]:
betano.drop(['Unnamed: 0', 'data', 'hora', 'casa_ganha', 'empate', 'visitante_ganha', 'simodd', 'nãoodd'], axis='columns', inplace=True)
betfair.drop(['Unnamed: 0','data/hora', 'casa_ganha', 'empate', 'visitante_ganha', 'empate', 'visitante_ganha'], axis='columns', inplace=True)
sporting.drop(['Unnamed: 0', 'aovivo','casa_ganha', 'empate', 'visitante_ganha'], axis='columns', inplace=True)

In [34]:
betano.head()

,time_casa,time_visitante,maisq,menosq
0,Club Atletico Tucuman II,Boca Juniors II,1.98,1.65
1,CF Tigres UANL (F),Deportivo Toluca FC (F),NaN,NaN
2,Once Caldas,Deportes Tolima,2.70,1.47
3,Mazatlán FC (F),Cruz Azul (F),1.52,2.20
4,Guadalajara Chivas (F),Juarez (F),1.60,2.05


In [35]:
betfair.head()

,maisq25,menosq25,time_casa,time_visitante
0,0,2.0,1.1,Roma
1,3,0.0,NaN,NaN
2,1,0.0,10.0,Sporting CP
3,0,2.0,1.001,Montpellier
4,2,1.0,Monaco,Auxerre


In [36]:
sporting.head()

,time_casa,time_visitante,mais/menosQ,maisQ,menos_que
0,Manchester United,Nottingham Forest,Aposte agora,NaN,NaN
1,Lens,Nice,11.50,3.50,1.40
2,Roma,Cremonese,18.50,7.25,1.12
3,Sporting,Braga,1.29,4.60,9.00
4,Botafogo,Nova Iguaçu,"2,5",2.05,1.68


In [41]:
# Filtrando a o número de gols 
sporting = sporting[sporting['mais/menosQ'] == '2,5']

In [42]:
sporting

,time_casa,time_visitante,mais/menosQ,maisQ,menos_que
4,Botafogo,Nova Iguaçu,"2,5",2.05,1.68
5,Athletico-PR,Azuriz,"2,5",1.78,1.90
6,Vila Nova,Grêmio Anápolis,"2,5",2.10,1.65
7,Bento Gonçalves-RS,Grêmio,"2,5",1.98,1.73
8,Cuiabá,Sport Sinop,"2,5",1.71,2.00
9,Fortaleza,Atlético-CE,"2,5",1.65,2.10
11,Barcelona de Ilhéus,Bahia,"2,5",2.25,1.57
12,Mirassol,Palmeiras,"2,5",2.15,1.61
14,Mirassol,Palmeiras,"2,5",2.15,1.61
16,Ferroviária,São Bernardo,"2,5",2.45,1.49


In [44]:
sporting['mais/menosQ'] = pd.to_numeric(sporting['mais/menosQ'], errors='coerce')
sporting['maisQ'] = pd.to_numeric(sporting['maisQ'], errors='coerce')
sporting['menos_que'] = pd.to_numeric(sporting['menos_que'], errors='coerce')

In [46]:
sporting

,time_casa,time_visitante,mais/menosQ,maisQ,menos_que
4,Botafogo,Nova Iguaçu,NaN,2.05,1.68
5,Athletico-PR,Azuriz,NaN,1.78,1.90
6,Vila Nova,Grêmio Anápolis,NaN,2.10,1.65
7,Bento Gonçalves-RS,Grêmio,NaN,1.98,1.73
8,Cuiabá,Sport Sinop,NaN,1.71,2.00
9,Fortaleza,Atlético-CE,NaN,1.65,2.10
11,Barcelona de Ilhéus,Bahia,NaN,2.25,1.57
12,Mirassol,Palmeiras,NaN,2.15,1.61
14,Mirassol,Palmeiras,NaN,2.15,1.61
16,Ferroviária,São Bernardo,NaN,2.45,1.49


In [47]:
betfair.head()

,maisq25,menosq25,time_casa,time_visitante
0,0,2.0,1.1,Roma
1,3,0.0,NaN,NaN
2,1,0.0,10.0,Sporting CP
3,0,2.0,1.001,Montpellier
4,2,1.0,Monaco,Auxerre


In [48]:
betfair['maisq25'] = pd.to_numeric(betfair['maisq25'], errors='coerce')
betfair['menosq25'] = pd.to_numeric(betfair['maisq25'], errors='coerce')

In [49]:
betano.head()

,time_casa,time_visitante,maisq,menosq
0,Club Atletico Tucuman II,Boca Juniors II,1.98,1.65
1,CF Tigres UANL (F),Deportivo Toluca FC (F),NaN,NaN
2,Once Caldas,Deportes Tolima,2.70,1.47
3,Mazatlán FC (F),Cruz Azul (F),1.52,2.20
4,Guadalajara Chivas (F),Juarez (F),1.60,2.05


In [50]:
betano['maisq'] = pd.to_numeric(betano['maisq'], errors='coerce')
betano['menosq'] = pd.to_numeric(betano['menosq'], errors='coerce')

In [55]:
# convertendo as colunas dos nomes do time para strings
betfair['time_casa'] = betfair.time_casa.astype(str)
betfair['time_visitante'] = betfair.time_visitante.astype(str)
betano['time_casa'] = betano.time_casa.astype(str)
sporting['time_casa'] = sporting.time_casa.astype(str)

In [59]:
# map the team names in betfair to match the names in betano and sporting
'''
df1['team1'] = df1['team1'].apply(lambda x: process.extractOne(x, df2['team_name'].tolist(), score_cutoff=60)[0] if process.extractOne(x, df2['team_name'].tolist(), score_cutoff=60) else x)
df1['team2'] = df1['team2'].apply(lambda x: process.extractOne(x, df3['team_name'].tolist(), score_cutoff=60)[0] if process.extractOne(x, df3['team_name'].tolist(), score_cutoff=60) else x)
'''
betfair['time_casa'] = betfair['time_casa'].apply(lambda x: process.extractOne(x, betano['time_casa'].tolist(), score_cutoff=60)[0] if process.extractOne(x, betano['time_casa'].tolist(), score_cutoff=60) else x)
betfair['time_visitante'] = betfair['time_visitante'].apply(lambda x: process.extractOne(x, sporting['time_casa'].tolist(), score_cutoff=60)[0] if process.extractOne(x, sporting['time_casa'].tolist(), score_cutoff=60) else x)

# merge the first and second dataframes on team1 and team_name
merged_df1 = pd.merge(betfair, betano, left_on='time_casa', right_on='time_casa')

# merge the merged dataframe with the third dataframe on team2 and team_name
merged_df2 = pd.merge(merged_df1, sporting, left_on='time_casa', right_on='time_casa')

# drop the redundant columns
#merged_df2.drop(['time_casa1', 'time_casa2', 'time_casa_x', 'time_casa_y'], axis=1, inplace=True)
merged_df2

,maisq25,menosq25,time_casa,time_visitante_x,time_visitante_y,maisq,menosq,time_visitante,mais/menosQ,maisQ,menos_que
